In [2]:
### 로컬 데이터 불러오기 ###
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'. format(name = fn, length = len(uploaded[fn])))

Saving infert.csv to infert.csv
User uploaded file "infert.csv" with length 6964 bytes


In [0]:
### 모듈 및 함수 로딩 ###
import io
import pandas as pd
import numpy as np
import tensorflow as tf
import random

In [0]:
infert = pd.read_csv(io.StringIO(uploaded['infert.csv'].decode('utf-8')))

In [6]:
infert.head()

,education,age,parity,induced,case,spontaneous,stratum,pooled.stratum
0,0-5yrs,26,6,1,1,2,1,3
1,0-5yrs,42,1,1,1,0,2,1
2,0-5yrs,39,6,2,1,0,3,4
3,0-5yrs,34,4,2,1,0,4,2
4,6-11yrs,35,3,1,1,1,5,32


In [0]:
X_data = infert[["parity", "induced", "spontaneous"]]
y_data = infert["case"]

In [0]:
tf.set_random_seed(123)
train_idx = random.sample(list(range(len(infert))), int(len(infert) * 0.7 ))

train_X = X_data.iloc[train_idx]
train_y = y_data[train_idx]

test_X = X_data.drop(train_idx)
test_y = y_data.drop(train_idx)

In [0]:
X = tf.placeholder(shape = (None, 3), dtype = tf.float32)
y = tf.placeholder(shape = (None), dtype = tf.float32)

W1 = tf.Variable(initial_value= tf.random_normal([3, 1]), dtype = tf.float32)
b1 = tf.Variable(initial_value= tf.random_normal([1]), dtype = tf.float32)
L1 = tf.nn.relu(tf.add(tf.matmul(X, W1), b1))

In [0]:
W2 = tf.Variable(initial_value= tf.random_normal([1, 1]), dtype = tf.float32)
b2 = tf.Variable(initial_value= tf.random_normal([1]), dtype = tf.float32)
hypo = tf.add(tf.matmul(L1, W2), b2)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = y, logits = hypo))
optimizer = tf.train.GradientDescentOptimizer(learning_rate= 0.0001)
train = optimizer.minimize(cost)

In [0]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [143]:
for i in range(100):
  _, cost_val = sess.run([train, cost], feed_dict = {X: train_X, y: train_y})
  if i % 10 ==0:
    print("step: ", i, "\t", "cost: ", cost_val)
print("train finished!!")

step:  0 	 cost:  309.19745
step:  10 	 cost:  309.19745
step:  20 	 cost:  309.19745
step:  30 	 cost:  309.19745
step:  40 	 cost:  309.19745
step:  50 	 cost:  309.19745
step:  60 	 cost:  309.19745
step:  70 	 cost:  309.19745
step:  80 	 cost:  309.19745
step:  90 	 cost:  309.19745
train finished!!


In [144]:
predict = tf.cast(hypo > 0.5, dtype = tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predict, y), dtype = tf.float32 ))
print("accuracy: ", sess.run(accuracy, feed_dict = {X: test_X, y: test_y}))

accuracy:  0.6933333
